In [1]:
import glob

In [2]:
from transformers import GPT2Tokenizer, GPTNeoForCausalLM,GPTNeoModel

In [6]:
from transformers import AutoModelForSequenceClassification

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M",    
                            bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>",truncation=True,padding=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("results/checkpoint-140000/")

In [7]:
# AutoModelForSequenceClassification.from_pretrained("results/checkpoint-140000/")

In [8]:
# model

In [5]:
model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0, inplace=False)
            (resid_dropout): Dropout(p=0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3072, o

In [10]:
import json

In [11]:
data = open("files_upload/data_test_prepared.jsonl","r").readlines()

In [12]:
data = [json.loads(i) for i in data]

In [13]:
data[0]

{'prompt': "GOAL α : Type u,\t_inst_1 : inhabited α,\tb : buffer α,\ti : ℕ,\th : i < b.size\t⊢ b.read ⟨i, h⟩ = b.read' i\n PROOFSTEP ",
 'completion': " cases b; unfold read read'; simp [array.read_eq_read']\n"}

In [38]:
len(hard)

NameError: name 'hard' is not defined

In [15]:
all_d = ['<|startoftext|>' + ' <GOAL> ' + i["prompt"].replace("GOAL","").replace("PROOFSTEP","") +  ' <PROOFSTEP>  ' for i in data]


In [16]:
import numpy as np
import torch

In [51]:
results = []
hard = []
model.cuda()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
all_d[0]

tr = tokenizer(all_d[0], return_tensors="pt",padding=True).input_ids

In [18]:
tr.shape

torch.Size([1, 74])

In [19]:
output = model(tr)

In [20]:
model

GPTNeoModel(
  (wte): Embedding(50259, 768)
  (wpe): Embedding(2048, 768)
  (drop): Dropout(p=0, inplace=False)
  (h): ModuleList(
    (0): GPTNeoBlock(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPTNeoAttention(
        (attention): GPTNeoSelfAttention(
          (attn_dropout): Dropout(p=0, inplace=False)
          (resid_dropout): Dropout(p=0, inplace=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPTNeoMLP(
        (c_fc): Linear(in_features=768, out_features=3072, bias=True)
        (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        (act): NewGELUActivation()
        (drop

In [21]:
logits = output[0]
hidden_states = output[1]

In [37]:
torch.mean(logits,axis=1)

tensor([[-6.5701e-01, -5.5526e-01,  6.5292e-01, -4.7964e-01,  1.2447e-01,
         -4.4783e-01, -3.2223e-01,  6.3936e-01, -1.1968e+00,  4.0471e-01,
          2.6583e-01, -6.3809e-01, -1.5580e+00,  4.7649e-02, -7.8752e-01,
         -9.3344e-02,  7.0465e-01,  1.1290e+00, -2.3664e-01, -1.2280e+00,
          4.5062e-01,  3.9161e-01, -8.2371e-01,  8.9174e-01, -8.8534e-01,
         -1.9812e+00,  4.1720e-01, -1.5243e-01, -6.0353e-02,  6.7547e-01,
         -8.3931e-01, -4.6398e-01,  3.8069e-01,  6.3796e-01, -5.6777e-01,
          2.2866e-01,  5.9045e-01,  6.4310e-02, -4.0508e-01, -1.2508e-01,
          1.1214e+00, -4.0432e-01,  8.7023e-01,  2.7850e-01,  1.3989e+00,
         -2.0679e-02, -3.5790e-01,  1.1951e-01,  4.1695e-01,  6.1825e-01,
          3.7406e-01,  7.5965e-01, -6.4549e-01,  9.9186e-01,  2.6553e-01,
         -4.4531e-02,  8.1131e-01, -6.3210e-01,  2.5568e-01,  1.3212e+00,
         -4.6501e-01,  5.4562e-01,  2.6294e-01,  4.9838e-01,  9.4803e-01,
          3.5791e+00, -9.8433e-01,  1.

In [29]:
# tensor.mean(logits)

In [ ]:
gen_tokens = model.generate(
                tr,
                do_sample=True,
                temperature=0.9,
                max_length=400, 
                num_return_sequences=15, 
            )
        

In [17]:
results

[]

In [18]:
hard = []

In [19]:
for i,j in zip(all_d,data):
    tr = tokenizer(i, return_tensors="pt",padding=True).input_ids
    try:
        tr = tr.to(device)
        gen_tokens = model.generate(
                tr,
                do_sample=True,
                temperature=0.9,
                max_length=400, 
                num_return_sequences=15, 
            )
        gen_text = tokenizer.batch_decode(gen_tokens)
        results.append((gen_text,j))
    except:
        hard.append(j)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 504, but ``max_length`` is set to 400. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 517, but ``max_length`` is set to 400. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 530, but ``max_length`` is set to 400. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 544, but ``max_length`` is set to 400. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
len(hard),len(results)

In [21]:
i = hard[99]

In [27]:
model.to(torch.device("cpu"))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [22]:
tr = '<|startoftext|>' + ' <GOAL> ' + i["prompt"].replace("GOAL","").replace("PROOFSTEP","") +  ' <PROOFSTEP>  '

In [23]:
tr = tokenizer(tr, return_tensors="pt",padding=True).input_ids

In [24]:
gen_tokens = model.generate(
                tr,
                do_sample=True,
                temperature=0.9,
                max_length=500, 
                num_return_sequences=15, 
            )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [56]:
gen_text = tokenizer.batch_decode(gen_tokens)

In [57]:
gen_text

['<|startoftext|>  <GOAL>  𝕜 : Type u_1,\t_inst_1 : nondiscrete_normed_field 𝕜,\tE : Type u_2,\t_inst_2 : normed_group E,\t_inst_3 : normed_space 𝕜 E,\tF : Type u_3,\t_inst_4 : normed_group F,\t_inst_5 : normed_space 𝕜 F,\tx : E,\ts : set E,\tt : set F,\ty : F,\tht : y ∈ closure t,\tv : E,\tc : ℕ → 𝕜,\td : ℕ → E,\thd : ∀ᶠ (n : ℕ) in filter.at_top, x + d n ∈ s,\thc : filter.tendsto (λ (n : ℕ), ∥c n∥) filter.at_top filter.at_top,\thy : filter.tendsto (λ (n : ℕ), c n • d n) filter.at_top (𝓝 v),\tn : ℕ,\tz : F,\thz : z ∈ {x : F | (λ (y_1 : F), ∥c n • (y_1 - y)∥ < (1 / 2) ^ n) x},\thzt : z ∈ t\t⊢ y + (z - y) ∈ t\n   <PROOFSTEP>   bound',
 '<|startoftext|>  <GOAL>  𝕜 : Type u_1,\t_inst_1 : nondiscrete_normed_field 𝕜,\tE : Type u_2,\t_inst_2 : normed_group E,\t_inst_3 : normed_space 𝕜 E,\tF : Type u_3,\t_inst_4 : normed_group F,\t_inst_5 : normed_space 𝕜 F,\tx : E,\ts : set E,\tt : set F,\ty : F,\tht : y ∈ closure t,\tv : E,\tc : ℕ → 𝕜,\td : ℕ → E,\thd : ∀ᶠ (n : ℕ) in filter.at_top, x + d n ∈

In [52]:
hard2 = []

In [40]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

In [41]:
model = model.to(device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [55]:
for i in hard:
#     try:
        
        tr = '<|startoftext|>' + ' <GOAL> ' + i["prompt"].replace("GOAL","").replace("PROOFSTEP","") +  ' <PROOFSTEP>  '
        tr = tokenizer(tr, return_tensors="pt",padding=True).input_ids
#         tr = tr.to(device)
        gen_tokens = model.generate(
                tr,
                do_sample=True,
                temperature=0.9,
                max_length=100, 
                num_return_sequences=10, 
            )
        gen_text = tokenizer.batch_decode(gen_tokens)
        results.append((gen_text,i))
#     except:
#         hard2.append(i)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 2792, but ``max_length`` is set to 100. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


IndexError: index out of range in self

In [39]:
len(hard)

6369

In [41]:
hard[0]

["<|startoftext|> <GOAL>  α : Type u,\t_inst_1 : inhabited α,\tb : buffer α,\ti : ℕ,\th : i < b.size\t⊢ b.read ⟨i, h⟩ = b.read' i\n   <PROOFSTEP>  ",
 "<|startoftext|> <GOAL>  α : Type u,\t_inst_1 : inhabited α,\tb : buffer α,\ti : ℕ,\th : i < b.size\t⊢ b.read ⟨i, h⟩ = b.read' i\n   <PROOFSTEP>  ",
 "<|startoftext|> <GOAL>  α : Type u,\t_inst_1 : inhabited α,\ti b_fst : ℕ,\tb_snd : array b_fst α,\th : i < buffer.size ⟨b_fst, b_snd⟩\t⊢ buffer.read ⟨b_fst, b_snd⟩ ⟨i, h⟩ = buffer.read' ⟨b_fst, b_snd⟩ i\n   <PROOFSTEP>  ",
 "<|startoftext|> <GOAL>  α : Type u,\t_inst_1 : inhabited α,\ti b_fst : ℕ,\tb_snd : array b_fst α,\th : i < buffer.size ⟨b_fst, b_snd⟩\t⊢ b_snd.read ⟨i, h⟩ = b_snd.read' i\n   <PROOFSTEP>  ",
 "<|startoftext|> <GOAL>  α : Type u,\t_inst_1 : inhabited α,\tb : buffer α,\ti : ℕ,\th : i < b.size\t⊢ b.read ⟨i, h⟩ = b.read' i\n   <PROOFSTEP>  ",
 '<|startoftext|> <GOAL>  α : Type u,\tf : α → α,\ta : α\t⊢ stream.iterate f a = a::stream.iterate f (f a)\n   <PROOFSTEP>  ',
 '<|s

In [248]:
for i in gen_text:
    print(i.split("<PROOFSTEP>")[-1].replace("<|endoftext|>",""))

   derangements

   substI F

   apply funext

   skip

   rw id_map

   refl

  dest : id ∘ funext rfl = funek

   refine funext (λ X, _)

   refl

   dsimp [map]



In [ ]:
1+1

In [103]:
gen_text = tokenizer.batch_decode(gen_tokens)
gen_text

["<|startoftext|>  <GOAL> GOAL α : Type u,\tb : buffer α,\ti : ℕ,\th : i < b.size,\tv : α\t⊢ b.write ⟨i, h⟩ v = b.write' i v\n PROOFSTEP  <PROOFSTEP>  refl\n<|endoftext|>"]

In [140]:
ls files_upload/data_test.jsonl

files_upload/data_test.jsonl


In [143]:
import json

with open('files_upload/data_test.jsonl', 'r') as json_file:
    json_list = [json.loads(i) for i in list(json_file)]

In [144]:
json_list[0]

{'prompt': "GOAL α : Type u,\t_inst_1 : inhabited α,\tb : buffer α,\ti : ℕ,\th : i < b.size\t⊢ b.read ⟨i, h⟩ = b.read' i\n PROOFSTEP ",
 'completion': "cases b; unfold read read'; simp [array.read_eq_read']\n"}